In [7]:
import pandas as pd
import numpy as np
import json
import pyterrier as pt
from NIRfunction import *
if not pt.started():
    pt.init(version="snapshot")

#### Import data

In [17]:
train_query = get_query('../Project/NIR2022 dataset/train_query.csv')
train_qrel = get_qrel('../Project/NIR2022 dataset/train_qrel.csv')
# corpus_pd = get_dataframe("../Project/NIR2022 dataset/corpus.jsonl")
files = pt.io.find_files("../Project/Data")

##### Different indexer

###### without pre-processing

In [ ]:
# build the index
indexer_TREC = pt.TRECCollectionIndexer("./indexes/stage1/index_TREC",  verbose=True, blocks=False, overwrite=True) 
indexer_TREC.setProperties(**{"termpipelines":""})
index_ref_TREC = indexer_TREC.index(files)

# Or load from files
# index_ref_TREC = pt.IndexRef.of("./indexes/stage1/index_TREC")

# load the index, print the statistics
index_TREC = pt.IndexFactory.of(index_ref_TREC)
print(index_TREC.getCollectionStatistics().toString())

 21%|██▏       | 492/2296 [01:03<03:06,  9.67files/s]

21:48:10.891 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in ../Project/Data/FR94/.DS_Store. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed


2297files [04:19,  8.86files/s]                       


Number of documents: 528155
Number of terms: 840517
Number of postings: 125861126
Number of fields: 0
Number of tokens: 269905822
Field names: []
Positions:   false



Pre-processing

In [ ]:
# build the index
indexer_TREC_stemmed = pt.TRECCollectionIndexer("./indexes/stage1/index_TREC_stemmed",  verbose=True, blocks=False, overwrite=True) 
indexer_TREC_stemmed.setProperties(**{"termpipelines":"EnglishSnowballStemmer, Stopwords"})
index_ref_TREC_stemmed = indexer_TREC_stemmed.index(files)

# Or load from files
# index_ref_TREC_stemmed = pt.IndexRef.of("./indexes/stage1/index_TREC_stemmed")

# load the index, print the statistics
index_TREC_stemmed = pt.IndexFactory.of(index_ref_TREC_stemmed)
print(index_TREC_stemmed.getCollectionStatistics().toString())

 21%|██▏       | 493/2296 [01:24<04:36,  6.53files/s]

21:53:14.590 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in ../Project/Data/FR94/.DS_Store. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed


2297files [05:33,  6.88files/s]                       


Number of documents: 528155
Number of terms: 739349
Number of postings: 93025387
Number of fields: 0
Number of tokens: 159579026
Field names: []
Positions:   false



##### indexer contain term position information

In [ ]:
# build the index
indexer_TREC_block = pt.TRECCollectionIndexer("./indexes/stage2/index_pd_block",  verbose=True, blocks=True, overwrite= True) 

indexer_TREC_block.setProperties(**{"termpipelines":"EnglishSnowballStemmer, Stopwords"})
index_ref_TREC_block = indexer_TREC_block.index(files)

# Or load from files
# index_ref_TREC_test = pt.IndexRef.of("./indexes/stage2/index_pd_block")

# load the index, print the statistics
index_TREC_block = pt.IndexFactory.of(index_ref_TREC_block)
print(index_TREC_block.getCollectionStatistics().toString())

 21%|██▏       | 493/2296 [01:33<04:55,  6.10files/s]

21:31:24.148 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in ../Project/Data/FR94/.DS_Store. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed


2297files [06:08,  6.24files/s]                       


Number of documents: 528155
Number of terms: 739349
Number of postings: 93025387
Number of fields: 0
Number of tokens: 159579026
Field names: []
Positions:   true



In [16]:
BM25_pd_block = pt.BatchRetrieve(index_TREC_block, wmodel="BM25")
BM25_pd_stemmed = pt.BatchRetrieve(index_TREC_stemmed, wmodel="BM25")
BM25_pd = pt.BatchRetrieve(index_TREC, wmodel="BM25")


pt.Experiment([BM25_pd_block, BM25_pd_stemmed, BM25_pd], 
                train_query, 
                train_qrel, 
                eval_metrics=["P_10", "map", "ndcg_cut_5", "ndcg_cut_10", "ndcg_cut_20", "mrt"],
                names=["Pre-processed with Term Positions ", "Pre-processed", "Without pre-processed"])

,name,P_10,map,ndcg_cut_5,ndcg_cut_10,ndcg_cut_20,mrt
0,Pre-processed with Term Positions,0.431658,0.248283,0.461505,0.442638,0.415147,21.157651
1,Pre-processed,0.431658,0.248283,0.461505,0.442638,0.415147,17.531459
2,Without pre-processed,0.384422,0.213686,0.424357,0.398934,0.380169,17.771872
